In [8]:
dataset = load_data('XRP', 1, 1000)

In [18]:
col=[6]
meta, data, _ = feed_maker(dataset, col, scale=False)

In [20]:
len(data)

1000

In [10]:
def prepare_data(n_in, n_out, cardinality, n_samples):
    

(1000, 11)

In [1]:
# 피쳐 계산? 예를 들어 트레이닝 기간 내 [평균가격/최근 n개 가격]이 529 라고 하면 언더 100 오버 100 정도 잡고 패딩의 민/맥스가 각 패딩의 50% 정도 되는 지점에 (예를 들어 +50) 다시 업데이트
# source / target / target_in 
# 데이터 가로로 쭉 줄세우고 10개씩 index 0 부터 +1 반복해서 가져온 뒤에, 앞에서 5개 끊어서 소스/ 뒤에서 5개 끊어서 타겟으로 만듬
# 어떻게 피쳐 업데이트할건지에 대한 고민 (o)

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import requests, json
from urllib.parse import quote

In [ ]:
#데이터셋 생성
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        target = list(np.arange(source[0] + 5,source[0]+n_out+5))
#         print(source)
#         print(target)
        
        target_in = [0] + target[:-1]
#         print('target:',target)
#         print('target-1:',target[:-1])
#         print('target_in:',target_in)
        
        # encode
        src_encoded = to_categorical(source, num_classes=cardinality)
        tar_encoded = to_categorical(target, num_classes=cardinality)
        tar2_encoded = to_categorical(target_in, num_classes=cardinality)
        
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return np.array(X1), np.array(X2), np.array(y)

#원핫 디코딩
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

#모델 생성
def define_models(n_input, n_output, n_units):
    
    #인코더 정의
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    
    #디코더 정의
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    # return all models
    return model, encoder_model, decoder_model

# 시퀀스 프레딕트
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return np.array(output)

In [15]:
#old functions
def call_candle(currency, minutes, count, to=''):
    max_count = 100

    if count > max_count:
        iter_range = count // max_count
        iter_left = count % max_count
        total = []

        while iter_range:
            to_encoded = quote(to)
            URL = 'https://api.upbit.com/v1/candles/minutes/' + str(
                minutes) + '?market=KRW-' + currency + '&count=' + str(max_count) + '&to=' + to_encoded
            req = requests.get(URL)
            res = json.loads(req.text)

            iter_range -= 1
            to = res[-1]['candle_date_time_kst'] + '+09:00'
            total += res

        if iter_left > 0:
            to_encoded = quote(to)
            URL = 'https://api.upbit.com/v1/candles/minutes/' + str(
                minutes) + '?market=KRW-' + currency + '&count=' + str(iter_left) + '&to=' + to_encoded
            req = requests.get(URL)
            res = json.loads(req.text)
            total += res

        return total
    else:
        to_encoded = quote(to)
        URL = 'https://api.upbit.com/v1/candles/minutes/' + str(
            minutes) + '?market=KRW-' + currency + '&count=' + str(count) + '&to=' + to_encoded
        req = requests.get(URL)
        res = json.loads(req.text)
        res.reverse()

        return res
    
def load_data(currencies, mins, counts):
    #loading data
    data_origin = call_candle(currencies, minutes=mins , count=counts, to='2018-11-11T23:00:00+09:00')
    data_array = []
    for i in data_origin:
        data_array.append(list(i.values()))
    data = np.array(data_array)
    data = data[data[:,2].argsort()] #sort by time
    return data

def feed_maker(data, col, scale=True, is_training=True, v=None):
    if is_training == True:
        v = {}
    x = {}
    
    #Missing Value 핸들링 : 각 값들을 median value로 채움
    if scale == True:
        for i in range(len(col)):
            if is_training == True:
                v[i] = get_metadata(data[:,col][:,i].astype(float))
            x[i] = scale_value(data[:,col][:,i].astype(float), v[i], scale=True)
    else:
        for i in range(len(col)):
            x[i] = data[:,col][:,i].astype(float)
    
    x_val = np.transpose(np.array(list(x.values())))
    y_val = x_val[:,[0]]
        
    return v, x_val, y_val

def get_metadata(feature):
    obj = {}
    obj['min'] = feature.min()
    obj['max'] = feature.max()
        
    return obj

def scale_value(feature, meta, scale):
    #수동 스케일러
    if scale == True:
        def minmaxScaler(data, datamin, datamax):
            numerator = data - datamin
            denominator = datamax - datamin
            return numerator / (denominator + 1e-8)
        
        return minmaxScaler(feature, meta['min'], meta['max'])

    else:
        pass